# Structured Output

The ability of LLMs to respond in structured format is extremely helpful for a variety of use cases such as when we want to call funtions with rigid parameters or update databases with defined schemas just to mention a few.

In graphs that pass state variables between nodes there are lots of different ways we can utilize structured outputs. 

## Using ToolNode

This example takes advantage of many pre-built Langchain functions that make adding an agent to your graph simple. First, we define our tools list and then create a node using the built in `ToolNode` functionality.

In [2]:
from langgraph.prebuilt import ToolNode
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=1)]
tool_node = ToolNode(tools)

We can then define a structured output we would like to return to our user, or perhaps another piece of our software for later use. In this case we will return weather data.

In [3]:
from langchain_core.pydantic_v1 import BaseModel, Field

class WeatherResponse(BaseModel):
    """Final response to the user"""

    temperature: float = Field(description="the temperature")
    wind_speed: float = Field(description="the wind speed")
    wind_direction: str = Field(description="the direction of the wind")

Let us define our model and bind our tools (including the `WeatherResponse`) to it.

In [4]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, streaming=True)
model = model.bind_tools(tools+[WeatherResponse])

We can now define the State class we will use for our graph, as well as the update function for our one member variables called `messages` which will just be a list of all the messages generated by our chain.

In [5]:
from typing import TypedDict, Annotated, Literal

def add_messages(left: list, right: list):
    """Add-don't-overwrite."""
    return left + right

class AgentState(TypedDict):
    # The `add_messages` function within the annotation defines
    # *how* updates should be merged into the state.
    messages: Annotated[list, add_messages]

We can now define the function for our model node, as well as the routing function to determine weather we need to call our tool.

In [13]:
def should_call_tool(state: AgentState) -> Literal["search", "__end__"]:
    # Check that we do indeed want to call our search tool
    if state['messages'][-1].tool_calls and state['messages'][-1].tool_calls[0]['name'] != "WeatherResponse":
        return "search"
    return "__end__"

def call_model(state: AgentState) -> AgentState:
    return {"messages":[model.invoke(state['messages'])]}

We are now ready to define our simple graph and compile it.

In [11]:
from langgraph.graph import StateGraph, END

graph = StateGraph(AgentState)
graph.add_node("agent", call_model)
graph.add_node("search", tool_node)
graph.set_entry_point("agent")
graph.add_conditional_edges("agent",should_call_tool)
graph.add_edge("search","agent")
app = graph.compile()

In [15]:
from langchain_core.messages import HumanMessage

ans = app.invoke({'messages':[HumanMessage(content="What is the weather in Barcelona?")]})
for message in ans['messages']:
    message.pretty_print()

================================ Human Message =================================

What is the weather in Barcelona?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_kGNgs8J6qnBRQdcfaF5CAm9z)
 Call ID: call_kGNgs8J6qnBRQdcfaF5CAm9z
  Args:
    query: weather in Barcelona
================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Barcelona', 'region': 'Catalonia', 'country': 'Spain', 'lat': 41.38, 'lon': 2.18, 'tz_id': 'Europe/Madrid', 'localtime_epoch': 1718076123, 'localtime': '2024-06-11 5:22'}, 'current': {'last_updated_epoch': 1718075700, 'last_updated': '2024-06-11 05:15', 'temp_c': 19.4, 'temp_f': 66.9, 'is_day': 0, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/night/122.png', 'code': 1009}, 'wind_mph': 3.8, 'wind_kph': 6.1, 'wind_degree': 330

When we run our model, we can see that it correctly executed the Tavily search as we expected, and in the last message it correctly returned data in the correct `WeatherResponse` structure.

## Executing Tools Manually

Sometime we have structured inputs to tools, but the LLM only provides a portion of that input and we need to manually input the data somehow before executing the tool.

Let us first define our tool which now takes two inputs, and create a binded model for our newly defined tool.

In [22]:
from langchain.tools import tool
from langgraph.prebuilt import ToolExecutor
from langchain_core.messages import ToolMessage

@tool
def search(query: str, max_results: int):
    '''Look things up online'''
    return [str(TavilySearchResults(max_results=max_results).invoke(query)).replace('\'','\"')]


tools = [search]
tool_executor = ToolExecutor(tools)
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, streaming=True)
model = model.bind_tools(tools)

Let's slightly edit our node function from above to accomodate this change in our tool definition.

In [23]:
from langgraph.prebuilt import ToolInvocation

def execute_search_with_user_input(state: AgentState):
    max_results = input(prompt="How many search results should we return? Enter a number 1-10")

    if max_results.isdigit() and int(max_results) > 0 and int(max_results) <= 10:
        last_message = state['messages'][-1]
        tool_call = last_message.tool_calls[0]
        action = ToolInvocation(
            tool=tool_call["name"],
            tool_input={**tool_call["args"],**{'max_results':int(max_results)}},
        )
        response = tool_executor.invoke(action)
        return {'messages':[ToolMessage(content=str(response),tool_call_id=tool_call['id'])]}
    else:
        raise ValueError

We can now compile the graph just like the example above.

In [24]:
graph = StateGraph(AgentState)
graph.add_node("agent", call_model)
graph.add_node("search", execute_search_with_user_input)
graph.set_entry_point("agent")
graph.add_conditional_edges("agent",should_call_tool)
graph.add_edge("search","agent")
app = graph.compile()

In [25]:
app.invoke({'messages':[HumanMessage(content="When are the 3 best movies of all time?")]})

{'messages': [HumanMessage(content='When are the 3 best movies of all time?'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_8qU9IsUtmU5c7itdOq1MUMyj', 'function': {'arguments': '{"query":"best movies of all time","max_results":3}', 'name': 'search'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-1dae89f3-ebec-4057-ad06-87d75a4fafb2-0', tool_calls=[{'name': 'search', 'args': {'query': 'best movies of all time', 'max_results': 3}, 'id': 'call_8qU9IsUtmU5c7itdOq1MUMyj'}]),
  ToolMessage(content='[\'[{"url": "https://www.imdb.com/chart/top/", "content": "You have rated\\\\nMore to explore\\\\nCharts\\\\nTop Box Office (US)\\\\nMost Popular Movies\\\\nTop Rated English Movies\\\\nMost Popular TV Shows\\\\nTop 250 TV Shows\\\\nLowest Rated Movies\\\\nMost Popular Celebs\\\\nTop Rated Movies by Genre\\\\nRecently viewed\\\\n© 1990-2024 by IMDb.com, Inc. The Lord of the Rings: The Return of the King\\\\n8. The Lord of th

In this example, we selected 3 search results, but on different runs the user will be able to select any amount of search results they want.

## Non-explicit Tools

Some schemas are not explicitly tools, but we can use them to make decisions in our graph, such as what node to proceed to next. Let us define a pydantic model that will inform our node selection.

In [28]:
class RoutingFunction(BaseModel):
    """Select the next node to proceed to"""

    next_node: str = Field(description="The next node to travel to. The options are: node_2 or node_3")

tool_node = ToolNode([RoutingFunction])

Let's bind this "function" to our model and force it to select the tool, by passing in the param `tool_choice`.

In [29]:
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, streaming=True)
model = model.bind_tools([RoutingFunction],tool_choice="RoutingFunction")

Now let's define our new graph, first specifying the functions we will need for each of our nodes.

In [63]:
from langchain_core.messages import SystemMessage

def node_2_func(state):
    return {"messages":[SystemMessage(content=f"I made it to node 2!")]}

def node_3_func(state):
    return {"messages":[SystemMessage(content=f"I made it to node 3!")]}

In [64]:
def select_which_node(state: AgentState):
    return state['messages'][-1].content.split('\'')[1]

In [65]:
graph = StateGraph(AgentState)
graph.add_node("agent", call_model)
graph.add_node("routing_tool", tool_node)
graph.add_edge("agent","routing_tool")
graph.add_node("node_2", node_2_func)
graph.add_node("node_3", node_3_func)
graph.set_entry_point("agent")
graph.add_conditional_edges("routing_tool",select_which_node)
graph.add_edge("node_2",END)
graph.add_edge("node_3",END)
app = graph.compile()

In [66]:
app.invoke({"messages":[HumanMessage(content="Please go to the node with the higher number")]})

{'messages': [HumanMessage(content='Please go to the node with the higher number'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_F2k4ZZxC1gHRv6bdNiRn4m4a', 'function': {'arguments': '{"next_node":"node_3"}', 'name': 'RoutingFunction'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'stop'}, id='run-f10b8b5e-5d41-43f1-b41c-9046d5bcb24d-0', tool_calls=[{'name': 'RoutingFunction', 'args': {'next_node': 'node_3'}, 'id': 'call_F2k4ZZxC1gHRv6bdNiRn4m4a'}]),
  ToolMessage(content="next_node='node_3'", name='RoutingFunction', tool_call_id='call_F2k4ZZxC1gHRv6bdNiRn4m4a'),
  SystemMessage(content='I made it to node 3!')]}

As we can see, by using structured output we can select the next node to travel to without having to explicitly state anything.